In [7]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime

In [8]:
df = pd.read_csv('../600K US Housing Properties.csv', low_memory=False)

In [9]:
df.head()

,property_url,property_id,address,street_name,apartment,city,state,latitude,longitude,postcode,...,property_type,property_status,year_build,total_num_units,listing_age,RunDate,agency_name,agent_name,agent_phone,is_owned_by_zillow
0,https://www.zillow.com/homedetails/3-Plat-83-1...,2064212272,"3 Plat #83-10, Wrangell, AK 99929",Plat #83-10,NaN,Wrangell,AK,NaN,NaN,99929,...,LOT,FOR_SALE,NaN,NaN,-1,2022-04-24 07:34:15,Anchor Properties,NaN,NaN,0
1,https://www.zillow.com/homedetails/117-3rd-St-...,249518113,"117 3rd St, Wrangell, AK 99929",3rd St,NaN,Wrangell,AK,56.474518,-132.38600,99929,...,SINGLE_FAMILY,FOR_SALE,NaN,NaN,-1,2022-04-24 07:34:15,NaN,NaN,NaN,0
2,https://www.zillow.com/homedetails/LOT-2A-Fron...,2077729913,"LOT 2A Front St, Wrangell, AK 99929",LOT 2A Front St,NaN,Wrangell,AK,56.471638,-132.38686,99929,...,LOT,FOR_SALE,NaN,NaN,-1,2022-04-24 07:34:15,NaN,NaN,NaN,0
3,https://www.zillow.com/homedetails/LOT-A-Plat-...,2067488502,"LOT A Plat #2009-6, Wrangell, AK 99929",LOT A Plat #2009-6,NaN,Wrangell,AK,NaN,NaN,99929,...,SINGLE_FAMILY,FOR_SALE,NaN,NaN,-1,2022-04-24 07:34:15,"Petersburg Properties, LLC",NaN,NaN,0
4,https://www.zillow.com/homedetails/335-Cassiar...,249518139,"335 Cassiar St, Wrangell, AK 99929",Cassiar St,NaN,Wrangell,AK,56.475697,-132.38905,99929,...,SINGLE_FAMILY,FOR_SALE,NaN,NaN,-1,2022-04-24 07:34:15,NaN,NaN,NaN,0


In [10]:
df.describe(include='all')

,property_url,property_id,address,street_name,apartment,city,state,latitude,longitude,postcode,...,property_type,property_status,year_build,total_num_units,listing_age,RunDate,agency_name,agent_name,agent_phone,is_owned_by_zillow
count,600000,6.000000e+05,600000,599869,14815,599999,599999,529122.000000,529122.000000,599970,...,600000,600000,0.0,0.0,600000.0,600000,444524,0.0,0.0,600000.000000
unique,600000,NaN,598588,339224,2664,7977,25,NaN,NaN,10820,...,7,2,NaN,NaN,NaN,1,34372,NaN,NaN,NaN
top,https://www.zillow.com/homedetails/3-Plat-83-1...,NaN,"(undisclosed Address), Rockport, TX 78382",(undisclosed Address),1,Chicago,TX,NaN,NaN,84043,...,SINGLE_FAMILY,FOR_SALE,NaN,NaN,NaN,2022-04-24 07:34:15,Coldwell Banker Realty,NaN,NaN,NaN
freq,1,NaN,36,1713,312,14138,146636,NaN,NaN,1102,...,354366,383365,NaN,NaN,NaN,600000,5936,NaN,NaN,NaN
mean,NaN,8.885042e+08,NaN,NaN,NaN,NaN,NaN,36.282379,-105.813906,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000498
std,NaN,9.724708e+08,NaN,NaN,NaN,NaN,NaN,5.673355,13.464633,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.022318
min,NaN,2.700000e+01,NaN,NaN,NaN,NaN,NaN,18.985142,-165.408250,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000000
25%,NaN,5.402143e+07,NaN,NaN,NaN,NaN,NaN,32.612112,-117.346079,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000000
50%,NaN,2.066090e+08,NaN,NaN,NaN,NaN,NaN,35.403568,-101.897378,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000000
75%,NaN,2.066867e+09,NaN,NaN,NaN,NaN,NaN,39.661674,-95.354245,NaN,...,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,0.000000


In [11]:
df.isna().sum()

property_url               0
property_id                0
address                    0
street_name              131
apartment             585185
city                       1
state                      1
latitude               70878
longitude              70878
postcode                  30
price                      0
bedroom_number        156155
bathroom_number       128267
price_per_unit        164635
living_space          152153
land_space             84881
land_space_unit        84881
broker_id             600000
property_type              0
property_status            0
year_build            600000
total_num_units       600000
listing_age                0
RunDate                    0
agency_name           155476
agent_name            600000
agent_phone           600000
is_owned_by_zillow         0
dtype: int64

In [12]:
df_isna=df.isna()
df_isna

,property_url,property_id,address,street_name,apartment,city,state,latitude,longitude,postcode,...,property_type,property_status,year_build,total_num_units,listing_age,RunDate,agency_name,agent_name,agent_phone,is_owned_by_zillow
0,False,False,False,False,True,False,False,True,True,False,...,False,False,True,True,False,False,False,True,True,False
1,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
2,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
3,False,False,False,False,True,False,False,True,True,False,...,False,False,True,True,False,False,False,True,True,False
4,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
599996,False,False,False,False,True,False,False,True,True,False,...,False,False,True,True,False,False,False,True,True,False
599997,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
599998,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False


In [13]:
df_isna.columns=[c for c in df_isna.columns]
df_isna

,property_url,property_id,address,street_name,apartment,city,state,latitude,longitude,postcode,...,property_type,property_status,year_build,total_num_units,listing_age,RunDate,agency_name,agent_name,agent_phone,is_owned_by_zillow
0,False,False,False,False,True,False,False,True,True,False,...,False,False,True,True,False,False,False,True,True,False
1,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
2,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
3,False,False,False,False,True,False,False,True,True,False,...,False,False,True,True,False,False,False,True,True,False
4,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
599996,False,False,False,False,True,False,False,True,True,False,...,False,False,True,True,False,False,False,True,True,False
599997,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
599998,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False


In [14]:
df_isna.columns=[c+'_isna' for c in df_isna.columns]
df_isna

,property_url_isna,property_id_isna,address_isna,street_name_isna,apartment_isna,city_isna,state_isna,latitude_isna,longitude_isna,postcode_isna,...,property_type_isna,property_status_isna,year_build_isna,total_num_units_isna,listing_age_isna,RunDate_isna,agency_name_isna,agent_name_isna,agent_phone_isna,is_owned_by_zillow_isna
0,False,False,False,False,True,False,False,True,True,False,...,False,False,True,True,False,False,False,True,True,False
1,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
2,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
3,False,False,False,False,True,False,False,True,True,False,...,False,False,True,True,False,False,False,True,True,False
4,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
599996,False,False,False,False,True,False,False,True,True,False,...,False,False,True,True,False,False,False,True,True,False
599997,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
599998,False,False,False,False,True,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False


In [15]:
df=pd.concat([df.fillna(0), df_isna.astype(float)], axis=1)
df

,property_url,property_id,address,street_name,apartment,city,state,latitude,longitude,postcode,...,property_type_isna,property_status_isna,year_build_isna,total_num_units_isna,listing_age_isna,RunDate_isna,agency_name_isna,agent_name_isna,agent_phone_isna,is_owned_by_zillow_isna
0,https://www.zillow.com/homedetails/3-Plat-83-1...,2064212272,"3 Plat #83-10, Wrangell, AK 99929",Plat #83-10,0,Wrangell,AK,0.000000,0.000000,99929,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,https://www.zillow.com/homedetails/117-3rd-St-...,249518113,"117 3rd St, Wrangell, AK 99929",3rd St,0,Wrangell,AK,56.474518,-132.386000,99929,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
2,https://www.zillow.com/homedetails/LOT-2A-Fron...,2077729913,"LOT 2A Front St, Wrangell, AK 99929",LOT 2A Front St,0,Wrangell,AK,56.471638,-132.386860,99929,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
3,https://www.zillow.com/homedetails/LOT-A-Plat-...,2067488502,"LOT A Plat #2009-6, Wrangell, AK 99929",LOT A Plat #2009-6,0,Wrangell,AK,0.000000,0.000000,99929,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,https://www.zillow.com/homedetails/335-Cassiar...,249518139,"335 Cassiar St, Wrangell, AK 99929",Cassiar St,0,Wrangell,AK,56.475697,-132.389050,99929,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,https://www.zillow.com/homedetails/6317-Virgin...,4303608,"6317 Virginia Ln, Matteson, IL 60443",Virginia Ln,0,Matteson,IL,41.523070,-87.769340,60443,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
599996,https://www.zillow.com/homedetails/LOT-16-Ray-...,2078144422,"LOT 16 Ray Ave, West Chicago, IL 60185",LOT 16 Ray Ave,0,West Chicago,IL,0.000000,0.000000,60185,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
599997,https://www.zillow.com/homedetails/12S135-Lemo...,4604875,"12S135 Lemont Rd, Lemont, IL 60439",135 Lemont Rd,0,Lemont,IL,41.697723,-88.003746,60439,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
599998,https://www.zillow.com/homedetails/4701-Abbey-...,65578365,"4701 Abbey Ln, Matteson, IL 60443",Abbey Ln,0,Matteson,IL,41.519527,-87.730644,60443,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [16]:
features=df.describe().columns
features

Index(['property_id', 'latitude', 'longitude', 'price', 'bedroom_number',
       'bathroom_number', 'price_per_unit', 'living_space', 'land_space',
       'broker_id', 'year_build', 'total_num_units', 'listing_age',
       'agent_name', 'agent_phone', 'is_owned_by_zillow', 'property_url_isna',
       'property_id_isna', 'address_isna', 'street_name_isna',
       'apartment_isna', 'city_isna', 'state_isna', 'latitude_isna',
       'longitude_isna', 'postcode_isna', 'price_isna', 'bedroom_number_isna',
       'bathroom_number_isna', 'price_per_unit_isna', 'living_space_isna',
       'land_space_isna', 'land_space_unit_isna', 'broker_id_isna',
       'property_type_isna', 'property_status_isna', 'year_build_isna',
       'total_num_units_isna', 'listing_age_isna', 'RunDate_isna',
       'agency_name_isna', 'agent_name_isna', 'agent_phone_isna',
       'is_owned_by_zillow_isna'],
      dtype='object')

In [17]:
targets=['price','price_per_unit']
targets


['price', 'price_per_unit']

In [18]:
features=[c for c in df.describe().columns if not c.lower().startswith('price')]
features

['property_id',
 'latitude',
 'longitude',
 'bedroom_number',
 'bathroom_number',
 'living_space',
 'land_space',
 'broker_id',
 'year_build',
 'total_num_units',
 'listing_age',
 'agent_name',
 'agent_phone',
 'is_owned_by_zillow',
 'property_url_isna',
 'property_id_isna',
 'address_isna',
 'street_name_isna',
 'apartment_isna',
 'city_isna',
 'state_isna',
 'latitude_isna',
 'longitude_isna',
 'postcode_isna',
 'bedroom_number_isna',
 'bathroom_number_isna',
 'living_space_isna',
 'land_space_isna',
 'land_space_unit_isna',
 'broker_id_isna',
 'property_type_isna',
 'property_status_isna',
 'year_build_isna',
 'total_num_units_isna',
 'listing_age_isna',
 'RunDate_isna',
 'agency_name_isna',
 'agent_name_isna',
 'agent_phone_isna',
 'is_owned_by_zillow_isna']